In [12]:
import pandas as pd
import os
pd.set_option('display.max_columns', 500)

In [13]:
DATA_FOLDER = './data/'

edges_csv = 'panama_papers.edges.csv'
intermediary_csv = 'panama_papers.nodes.intermediary.csv' # company or individuals
address_csv = 'panama_papers.nodes.address.csv'
officer_csv = 'panama_papers.nodes.officer.csv'
entity_csv = 'panama_papers.nodes.entity.csv' # tax heaven companies

# forbes
forbes_csv = 'Forbes2015.csv'

In [14]:
edges = pd.read_csv(DATA_FOLDER + edges_csv) # mixed columns
intermediary = pd.read_csv(DATA_FOLDER + intermediary_csv)
address = pd.read_csv(DATA_FOLDER + address_csv)
officer = pd.read_csv(DATA_FOLDER + officer_csv)
entity = pd.read_csv(DATA_FOLDER + entity_csv) # mixed columns

# forbes
forbes_2000 = pd.read_csv(DATA_FOLDER + forbes_csv)

/home/eddie/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/eddie/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


279627881

In [27]:
edges.shape

(674102, 8)

In [21]:
edges.loc[edges.START_ID == 15005001]

,START_ID,TYPE,END_ID,link,start_date,end_date,sourceID,valid_until
674071,15005001,officer_of,10162180,protector of,NaN,NaN,Panama Papers,The Panama Papers data is current through 2015


In [22]:
edges.loc[edges.END_ID == 10162180]

,START_ID,TYPE,END_ID,link,start_date,end_date,sourceID,valid_until
170221,11011539,intermediary_of,10162180,intermediary of,NaN,NaN,Panama Papers,The Panama Papers data is current through 2015
674071,15005001,officer_of,10162180,protector of,NaN,NaN,Panama Papers,The Panama Papers data is current through 2015
674073,15005002,officer_of,10162180,"beneficiary, shareholder and director of",NaN,NaN,Panama Papers,The Panama Papers data is current through 2015
674076,15005003,officer_of,10162180,"beneficiary, shareholder and director of",NaN,NaN,Panama Papers,The Panama Papers data is current through 2015
674078,15005004,officer_of,10162180,beneficiary of,NaN,NaN,Panama Papers,The Panama Papers data is current through 2015


In [24]:
entity#[entity.name == "LINK PACIFIC CORPORATION"]

,node_id,name,jurisdiction,jurisdiction_description,country_codes,countries,incorporation_date,inactivation_date,struck_off_date,closed_date,ibcRUC,status,company_type,service_provider,sourceID,valid_until,note
0,10000001,"TIANSHENG INDUSTRY AND TRADING CO., LTD.",SAM,Samoa,HKG,Hong Kong,23-MAR-2006,18-FEB-2013,15-FEB-2013,NaN,25221,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN
1,10000002,"NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.",SAM,Samoa,HKG,Hong Kong,27-MAR-2006,27-FEB-2014,15-FEB-2014,NaN,25249,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN
2,10000003,"HOTFOCUS CO., LTD.",SAM,Samoa,HKG,Hong Kong,10-JAN-2006,15-FEB-2012,15-FEB-2012,NaN,24138,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN
3,10000004,"SKY-BLUE GIFTS & TOYS CO., LTD.",SAM,Samoa,HKG,Hong Kong,06-JAN-2006,16-FEB-2009,15-FEB-2009,NaN,24012,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN
4,10000005,FORTUNEMAKER INVESTMENTS CORPORATION,SAM,Samoa,HKG,Hong Kong,19-APR-2006,15-MAY-2009,15-FEB-2008,NaN,R25638,Changed agent,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN
5,10000006,8808 HOLDING LIMITED,SAM,Samoa,HKG,Hong Kong,05-JAN-2006,NaN,NaN,NaN,23835,Active,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN
6,10000007,KENT DEVELOPMENT LIMITED,SAM,Samoa,HKG,Hong Kong,26-JAN-2004,03-MAY-2006,15-FEB-2006,NaN,15757,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN
7,10000008,BONUS TRADE LIMITED,SAM,Samoa,HKG,Hong Kong,13-FEB-2004,16-FEB-2009,15-FEB-2009,NaN,15910,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN
8,10000009,AMARANDAN LTD.,SAM,Samoa,HKG,Hong Kong,26-JAN-2004,03-MAY-2006,15-FEB-2006,NaN,15759,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN
9,10000010,NEW IDEA LIMITED,SAM,Samoa,HKG,Hong Kong,30-MAR-2004,27-FEB-2007,15-FEB-2007,NaN,16462,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN


In [25]:
inter

NameError: name 'inter' is not defined

In [6]:
intermediary.countries.unique()

array(['South Africa', 'Liechtenstein', 'Monaco', 'Belgium', 'Lebanon',
       'Switzerland', 'Malaysia', 'Spain', 'United Kingdom',
       'Jersey;United Kingdom', 'France', 'Luxembourg', 'Taiwan',
       'Estonia', 'Mexico', 'Argentina', 'Guernsey;United Kingdom',
       'United States', 'Venezuela', 'Hong Kong', 'Panama',
       'Saudi Arabia', 'Germany', nan, 'Kuwait', 'Poland', 'Brazil',
       'Turkey', 'Egypt', 'Canada', 'Portugal', 'Russia', 'Isle of Man',
       'Malta', 'Hungary', 'Israel', 'Greece', 'Philippines', 'Italy',
       'China', 'Gibraltar', 'Bahamas', 'Honduras', 'Australia',
       'Austria', 'Sweden', 'Slovenia', 'Uruguay', 'Panama;Spain',
       'Thailand', 'Ecuador', 'Colombia', 'United Arab Emirates', 'Peru',
       'Czech Republic', 'South Korea', 'Costa Rica', 'Andorra', 'Cyprus',
       'Cayman Islands', 'Latvia', 'Antigua and Barbuda', 'Guatemala',
       'Ukraine', 'New Zealand', 'Chile', 'El Salvador', 'Seychelles',
       'Uganda', 'Iceland', 'Bolivia'

In [1]:
entity[entity.countries == 'United States']

NameError: name 'entity' is not defined

In [10]:
address[address.countries == 'United States']

,node_id,name,address,country_codes,countries,sourceID,valid_until,note
90,14000091,NaN,#25 Mason Complex,USA,United States,Panama Papers,The Panama Papers data is current through 2015,NaN
211,14000212,NaN,% The Marvin M. Schwan Charitable Foundation; ...,USA,United States,Panama Papers,The Panama Papers data is current through 2015,NaN
224,14000225,NaN,@ Yunita; c/o Apt 501; 1201 Boylston Avenue; S...,USA,United States,Panama Papers,The Panama Papers data is current through 2015,NaN
256,14000257,NaN,020 ORIONCT; SANLEANDRO; CA 94579 USA,USA,United States,Panama Papers,The Panama Papers data is current through 2015,NaN
492,14006727,NaN,19 ANDALUCIA; IRVINE; CA 92614; USA,USA,United States,Panama Papers,The Panama Papers data is current through 2015,NaN
535,14006770,NaN,19 BRISTOL DRIVE NO BRUNSWICK NJ 08902.,USA,United States,Panama Papers,The Panama Papers data is current through 2015,NaN
538,14006773,NaN,19 BROTH BRUNSWICK NJ 08902.,USA,United States,Panama Papers,The Panama Papers data is current through 2015,NaN
543,14006778,NaN,"19 Canterbury CT chesapeake City, MD 219151835...",USA,United States,Panama Papers,The Panama Papers data is current through 2015,NaN
641,14006876,NaN,19 MOUNTAIN ROAD VERNON USA,USA,United States,Panama Papers,The Panama Papers data is current through 2015,NaN
672,14006907,NaN,19 SCOTT DRIVE MELVILLE NEW YORK 11747 USA,USA,United States,Panama Papers,The Panama Papers data is current through 2015,NaN


In [25]:
forbes_2000

,Company,Sector,Industry,Continent,Country,Market Value,Sales,Profits,Assets,Rank,Forbes Webpage
0,BHP Billiton,Materials,Diversified Metals & Mining,Australia,Australia,182.3,67.7,14.8,151.0,44,http://www.forbes.com/companies/bhp-billiton/
1,Commonwealth Bank,Financials,Major Banks,Australia,Australia,114.5,41.5,7.9,699.9,49,http://www.forbes.com/companies/commonwealth-b...
2,Westpac Banking Group,Financials,Major Banks,Australia,Australia,99.0,38.4,6.8,651.4,60,http://www.forbes.com/companies/westpac-banking/
3,National Australia Bank,Financials,Major Banks,Australia,Australia,75.3,36.9,5.4,755.9,74,http://www.forbes.com/companies/national-austr...
4,ANZ,Financials,Major Banks,Australia,Australia,83.9,34.0,6.2,659.7,75,http://www.forbes.com/companies/anz/
5,Wesfarmers,Consumer Staples,Food Retail,Australia,Australia,43.5,58.5,2.3,39.3,205,http://www.forbes.com/companies/wesfarmers/
6,Telstra,Telecommunication Services,Telecommunications services,Australia,Australia,58.1,24.7,3.8,35.6,225,http://www.forbes.com/companies/telstra/
7,Woolworths,Consumer Staples,Food Retail,Australia,Australia,41.3,58.3,2.3,21.7,284,http://www.forbes.com/companies/woolworths/
8,AMP,Financials,Investment Services,Australia,Australia,13.5,19.4,0.6,119.2,431,http://www.forbes.com/companies/amp/
9,Macquarie Group,Financials,Investment Services,Australia,Australia,17.2,10.3,0.9,144.6,432,http://www.forbes.com/companies/macquarie-group/
